In [1]:
import pandas as pd
import networkx as nx
import ast
import matplotlib.pyplot as plt
import numpy as np

In [139]:
friends_file_path_mask = "../Data/ProcesData/friends/users_from_%s_cluster_friends.csv"
user_info_file_path_mask = "../Data/ProcesData/users_vk_info/users_from_%s_cluster_info.csv"
social_graph_file_path_mask = "../Data/ResultData/graphs/raw_graphs/cluster_%s_social_graph.gexf"

In [168]:
cluster_list = [0,136,45, 53, 137, 123, 105, 129]

In [89]:
chekins = pd.read_csv('../Data/RawData/checkins.csv', encoding='windows-1251')

chekins_group_by_users = pd.DataFrame(chekins.groupby('uid').id.count())
chekins_group_by_users = chekins_group_by_users.rename(index=int, columns={"id": "checkins_count"})
chekins_group_by_users = chekins_group_by_users.reset_index()
checkins_group_by_users['uid'] = checkins_group_by_users.uid.apply(lambda uid: int(float(uid)))

In [ ]:
def get_nickname(row):
    if isinstance(row['nickname'], float):
        return row['first_name'] + " " + row['last_name']
    else:
        return row['nickname']

In [169]:
for cluster_num in cluster_list:
    
    user_friends_from_cluster_df = pd.read_csv(friends_file_path_mask % (cluster_num))
    user_friends_from_cluster_df = user_friends_from_cluster_df.drop(['Unnamed: 0'], axis=1)
    
    user_friends_from_cluster_df = user_friends_from_cluster_df.merge(checkins_group_by_users, on='uid', how='left')
    
    users_from_cluster_vk_info_df = pd.read_csv(user_info_file_path_mask % (cluster_num), encoding='windows-1251')
    users_from_cluster_vk_info_df = users_from_cluster_vk_info_df.drop(['Unnamed: 0', 'occupation', 'city', 'country'], axis=1)
    users_from_cluster_vk_info_df['nickname'] = users_from_cluster_vk_info_df.apply(get_nickname, axis=1)
    
    user_friends_from_cluster_df = user_friends_from_cluster_df.merge(users_from_cluster_vk_info_df, on='uid', how='left')
    
    user_friends_from_cluster_df['friends_list'] =  user_friends_from_cluster_df.friends_list.apply(lambda f_l: ast.literal_eval(f_l))

    user_friends_from_cluster_df = user_friends_from_cluster_df.set_index('uid')
    
    nodes = user_friends_from_cluster_df.drop(['friends_list'], axis=1).to_dict('index')
    
    G = nx.Graph()

    for node in nodes:
        G.add_node(node, nodes[node])
    
    for node in G.nodes():
        user_friends_list = user_friends_from_cluster_df.friends_list[node]
        
        for friend_node in user_friends_list:
            if (friend_node in nodes.keys()):
                G.add_edge(node, friend_node)
                
    nx.write_gexf(G, social_graph_file_path_mask % (cluster_num))
    